# ST2195 Coursework question 4

### UOL Student Number 200549079

# Importing & Loading Necessary Packages 

In [1]:
import os
import sqlite3
import pandas as pd

# Working Directory

In [2]:
# Input your own directory where the Harvard Dataverse files are located within the code below.
os.chdir("C:\\Users\\Sudharsaan\\OneDrive\\SIM\\Year 2\\ST2195 Programming for Data Science\\Coursework\\Essential Harvard Dataverse Files")
os.getcwd()

'C:\\Users\\Sudharsaan\\OneDrive\\SIM\\Year 2\\ST2195 Programming for Data Science\\Coursework\\Essential Harvard Dataverse Files'

# Establishing connection with airlinemain_py database

In [3]:
conn = sqlite3.connect("airlinemain_py.db")

In [4]:
c = conn.cursor()

# Performing Query on question 4 of coursework


Question 4 asks whether we are able to detect cascading failures as delays in one airport create delays in others. There are 2 perspectives we can implement to answer the question:

**1. Observe planes which have consecutive flights**
- We can identify whether a plane has departed late from an airport & reached late at its destination. Then, we can check if the plane has departed late from the destination airport to verify if there are cascading failures. 
- If the departure delay from the destination airport is greater than its arrival delay at the same destination airport, we can also verify that the delays are increasing as the plane travels from one location to other locations consecutively.

**2. Observe planes that arrive late & how they affect other planes' departure**
- We can identify a flight that has arrived late & take note of the time at which it arrives at the airport. Then, we identify planes that are scheduled to depart from the same airport within a certain timeframe from which the late flight arrived.
- We verify if those identified planes also depart late which could be attributed to the plane which reached the airport late. To further verify that the identified planes' late departure is due to the late arrival of the initial flight, we can check whether these late departed flights result in more flights departing late. 

## Answering question 4 using Perspective 1

The **LateAircraftDelay** variable in the ontime table refers to the delays created as a result of the previous flight's late arrival which requires the use of the same plane to depart from that airport. This in turn causes the current scheduled flight to take off late & creates a domino effect, which causes passengers at other airports to board the aircraft much later as well.

Thus, it makes sense to query aircrafts which have "LateAircraftDelay" to be able to check if there is indeed cascading failures as delays in one airport create delays in others. Take note that we query for LateAircraftDelay with the values between 0 & 100 mins as flights with moderate amounts of delay tend not to have their future fligths cancelled which makes analysis easier for us. 

In [5]:
# Querying flights that have experienced "LateAircraftDelay"
q4_lateaircraftdelay = c.execute('''
                                    SELECT Year, Month, DayofMonth, DepTime, CRSDepTime, ArrTime, CRSArrTime, TailNum, ArrDelay,
                                    DepDelay, Origin, Dest, LateAircraftDelay
                                    FROM ontime
                                    WHERE Cancelled = 0 AND Diverted = 0 AND LateAircraftDelay > 0 AND LateAircraftDelay < 100 
                                    AND TailNum != 0
                                    ORDER BY LateAircraftDelay DESC''')
q4_lateaircraftdelay = pd.DataFrame(q4_lateaircraftdelay)
q4_lateaircraftdelay = q4_lateaircraftdelay.rename(columns = {0: "Year", 1: "Month", 2: "DayofMonth", 3: "DepTime", 
                                                              4: "CRSDepTime", 5: "ArrTime", 6: "CRSArrTime", 7: "TailNum", 
                                                              8: "ArrDelay", 9: "DepDelay", 10: "Origin", 11: "Dest", 
                                                              12: "LateAircraftDelay"})
q4_lateaircraftdelay

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,TailNum,ArrDelay,DepDelay,Origin,Dest,LateAircraftDelay
0,2006,1,13,2013.0,1820,2202.0,2023,N165UW,99.0,113.0,LGA,CLT,99
1,2006,1,13,1937.0,1740,2211.0,2032,N441US,99.0,117.0,ORD,CLT,99
2,2006,1,23,1906.0,1650,2104.0,1925,N667AW,99.0,136.0,CLE,PHX,99
3,2006,1,23,1959.0,1820,2209.0,2026,N591US,103.0,99.0,EWR,CLT,99
4,2006,1,2,2255.0,2110,2249.0,2110,N663SW,99.0,105.0,BOI,GEG,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408602,2007,12,13,1755.0,1550,2233.0,1912,N3752,201.0,125.0,JFK,TPA,1
1408603,2007,12,14,1047.0,1015,1337.0,1304,N396DA,33.0,32.0,BOS,ATL,1
1408604,2007,12,14,1636.0,1626,2002.0,1928,N675DL,34.0,10.0,BOS,ATL,1
1408605,2007,12,14,1510.0,1446,1631.0,1614,N901DE,17.0,24.0,ATL,CMH,1


**Observations from table above**

From the table above, we can observe many flights (1.4 million approximately) that have "LateAircraftDelay" associated with them. We shall analyse the **1st three rows** to check whether delays in one airport result in delays in other airports as well. 

Take note that we need only detect cascading failures which implies that it is sufficient to detect one such case. But we choose to analyse the 1st three rows for good measure. 

### Analysing flight with tail number N165UW on 13th January 2006

In [6]:
# Quering information regarding flight N165UW  
q4_N165UW = c.execute('''
                         SELECT Year, Month, DayofMonth, DepTime, CRSDepTime, ArrTime, CRSArrTime, TailNum, ArrDelay,
                         DepDelay, Origin, Dest, LateAircraftDelay
                         FROM ontime
                         WHERE Year = 2006 AND Month = 1 AND DayofMonth = 13 AND TailNum = 'N165UW'
                         ORDER BY CRSDepTime''')
q4_N165UW = pd.DataFrame(q4_N165UW)
q4_N165UW = q4_N165UW.rename(columns = {0: "Year", 1: "Month", 2: "DayofMonth", 3: "DepTime", 4: "CRSDepTime", 5: "ArrTime", 
                                        6: "CRSArrTime", 7: "TailNum", 8: "ArrDelay", 9: "DepDelay", 10: "Origin", 11: "Dest", 
                                        12: "LateAircraftDelay"})
q4_N165UW

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,TailNum,ArrDelay,DepDelay,Origin,Dest,LateAircraftDelay
0,2006,1,13,758.0,800,1010.0,1002,N165UW,8.0,-2.0,LGA,CLT,0
1,2006,1,13,1731.0,1545,1930.0,1724,N165UW,126.0,106.0,CLT,LGA,106
2,2006,1,13,2013.0,1820,2202.0,2023,N165UW,99.0,113.0,LGA,CLT,99
3,2006,1,13,2250.0,2130,1.0,2249,N165UW,72.0,80.0,CLT,JAX,72


**Observations from table above**

Flight N165UW was scheduled to depart from Charlotte/Douglos International (CLT) airport at 1545hrs (as indicated by CRSDepTime) but left at 1731hrs (as indicated by DepTime). This resulted in a departure delay of 106 mins (as indicated by DepDelay). Flight N165UW was scheduled to arrive at LaGuardia (LGA) airport at 1724hrs (as indicated by CRSArrTime) but arrived at 1930hrs (as indicated by ArrTime). This resulted in a arrival delay of 126 mins (as indicated by ArrDelay). This trip was the **origin of the delay** as the flight failed to prepare for departure at its scheduled time. 

Following its delayed arrival at LGA airport. The flight was scheduled to leave from LGA airport at 1820hrs & arrive at CLT airport at 2023hrs. However, due to the delay earlier, it departed from LGA airport at 2013hrs & arrived at CLT airport at 2202hrs which resulted in a departure delay of 113 mins & an arrival delay of 99 mins. The flight was able to **reduce its arrival delay from 126 mins to 99 mins**. Thus, the delays persisted. 

Following the trip from LGA to CLT, flight N165UW was scheduled for another flight from CLT to JAX. It was scheduled to depart from CLT airport at 2130hrs & arrive at JAX airport at 2249hrs. Once again, due to the delay earlier, the flight departed at 2250hrs & arrived at 0001hrs. This reuslted in a departure delay of 80 mins & an arrival delay of 72 mins. The flight was once again able to **reduce its arrival delay from 99 mins to 72 mins**. Thus, the delays continue to persist but reduce over time. 

Thus, we are **able to detect cascading failures** as delays in one airport created delays in others. It is worth mentioning that whether or not these delays persist depends on how spaced the flights' scheduled trips are as flights which have more "break" time between consecutive trips will be able to make up for delays incurred by reducing their break time.   

### Analysing flight with tail number N441US on 13th January 2006

In [7]:
# Quering information regarding flight N441US  
q4_N441US = c.execute('''
                         SELECT Year, Month, DayofMonth, DepTime, CRSDepTime, ArrTime, CRSArrTime, TailNum, ArrDelay,
                         DepDelay, Origin, Dest, LateAircraftDelay
                         FROM ontime
                         WHERE Year = 2006 AND Month = 1 AND DayofMonth = 13 AND TailNum = 'N441US'
                         ORDER BY CRSDepTime''')
q4_N441US = pd.DataFrame(q4_N441US)
q4_N441US = q4_N441US.rename(columns = {0: "Year", 1: "Month", 2: "DayofMonth", 3: "DepTime", 4: "CRSDepTime", 5: "ArrTime", 
                                        6: "CRSArrTime", 7: "TailNum", 8: "ArrDelay", 9: "DepDelay", 10: "Origin", 11: "Dest", 
                                        12: "LateAircraftDelay"})
q4_N441US

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,TailNum,ArrDelay,DepDelay,Origin,Dest,LateAircraftDelay
0,2006,1,13,627.0,620,905.0,740,N441US,85.0,7.0,SYR,DCA,0
1,2006,1,13,959.0,830,1232.0,1056,N441US,96.0,89.0,DCA,TPA,75
2,2006,1,13,1315.0,1150,1455.0,1327,N441US,88.0,85.0,TPA,CLT,77
3,2006,1,13,1823.0,1550,1904.0,1656,N441US,128.0,153.0,CLT,ORD,0
4,2006,1,13,1937.0,1740,2211.0,2032,N441US,99.0,117.0,ORD,CLT,99


**Observations from table above**

Similar to Flight N165UW, flight N441US sees its arrival & departure delay persist throughout all of its consecutive trips on 13th January 2006. Thus, we are able to detect another instance of cascading failure as delays in one airport create delays in other airports. 

### Analysing flight with tail number N667AW on 23rd January 2006

In [8]:
# Quering information regarding flight N667AW  
q4_N667AW = c.execute('''
                         SELECT Year, Month, DayofMonth, DepTime, CRSDepTime, ArrTime, CRSArrTime, TailNum, ArrDelay,
                         DepDelay, Origin, Dest, LateAircraftDelay
                         FROM ontime
                         WHERE Year = 2006 AND Month = 1 AND DayofMonth = 23 AND TailNum = 'N667AW'
                         ORDER BY CRSDepTime''')
q4_N667AW = pd.DataFrame(q4_N667AW)
q4_N667AW = q4_N667AW.rename(columns = {0: "Year", 1: "Month", 2: "DayofMonth", 3: "DepTime", 4: "CRSDepTime", 5: "ArrTime", 
                                        6: "CRSArrTime", 7: "TailNum", 8: "ArrDelay", 9: "DepDelay", 10: "Origin", 11: "Dest", 
                                        12: "LateAircraftDelay"})
q4_N667AW 

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,TailNum,ArrDelay,DepDelay,Origin,Dest,LateAircraftDelay
0,2006,1,23,817.0,818,1003.0,953,N667AW,10.0,-1.0,PHX,SLC,0
1,2006,1,23,1307.0,956,1831.0,1600,N667AW,151.0,191.0,PHX,CLE,0
2,2006,1,23,1041.0,1033,1216.0,1213,N667AW,3.0,8.0,SLC,PHX,0
3,2006,1,23,1906.0,1650,2104.0,1925,N667AW,99.0,136.0,CLE,PHX,99


In [9]:
# Subsetting appropriate rows required for analysis. 
q4_N667AW = q4_N667AW.drop([0,2])
q4_N667AW

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,TailNum,ArrDelay,DepDelay,Origin,Dest,LateAircraftDelay
1,2006,1,23,1307.0,956,1831.0,1600,N667AW,151.0,191.0,PHX,CLE,0
3,2006,1,23,1906.0,1650,2104.0,1925,N667AW,99.0,136.0,CLE,PHX,99


**Observations from table above**

Similar to Flights N165UW & N441US, flight N667AW also experiences persistent departure & arrival delays throughout its consecutive trips on 23rd January 2006. Thus, we detect a 3rd instance of cascading failures as delays in one aiport create delays in other airports. 

# Conclusion

From the observations made above, we are indeed **able to detect cascading failures** as delays in one aiport create delays in others. 

In [10]:
# Closing connection to airlinemain_py.db
conn.close()